# What is about ?

Get info on the genes used in MOA kaggle competition by MYGENE Python package.
See https://www.kaggle.com/alexandervc/genes-info-with-mygene-python-package
for info on that package.



In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Load data with Entrez id provided by competition organizers after the end of the competition

In [ ]:
url = 'https://github.com/LISHarvard/moa_challenge/blob/master/gene_mapping.csv'
url = 'https://raw.githubusercontent.com/LISHarvard/moa_challenge/master/gene_mapping.csv'
# Official link taken from: https://www.kaggle.com/c/lish-moa/discussion/207259

# Old - Entrez ID, New - used in competition

df = pd.read_csv(url, sep = ',')
df

# MYGENE package install and simple use example

In [ ]:
pip install mygene


In [ ]:
import mygene

In [ ]:
mg = mygene.MyGeneInfo()


In [ ]:
i = 0
g = df.iloc[i,0]

g = mg.getgene(g)#
g.keys()

In [ ]:
g[ 'symbol']

In [ ]:
g['type_of_gene']

In [ ]:
g['alias']

# Get names of all genes

In [ ]:
import time
list_genes_symbols  = []#  df.copy()
t0 = time.time()
for i in range(len(df)):
    entrez_gene_ID = df['old'].iat[i]
    kaggle_id = df['new'].iat[i]
    try:
        g_inf = mg.getgene(entrez_gene_ID)
        list_genes_symbols.append(g_inf['symbol'])
    except:
        print('Exception', kaggle_id,entrez_gene_ID )
        
print( 'Finished. Seconds passed: ',  np.round(time.time() - t0 ,1))
print( list_genes_symbols )        

# Get info on all genes from the list and save to  csv file

In [ ]:
import time
genes = df.copy()
t0 = time.time()
for i in range(len(df)):
    entrez_gene_ID = df['old'].iat[i]
    kaggle_id = df['new'].iat[i]
    
    try:
        g = mg.getgene(entrez_gene_ID)
        #print(i,ens_id)
        #print(g.keys())

        IX = genes.index[i]
        list1 = ['symbol',   'type_of_gene', 'map_location' ,   'name', 'entrezgene', 'alias', 'other_names'  ] 
        for k in list1:
            if k in g.keys():
                genes.loc[IX,k] = str(g[k])
        if  ('genomic_pos_hg19' in g.keys()) and ( isinstance(g['genomic_pos_hg19'], dict) ):
            for k in ['chr','start','end', 'strand' ]:
                if k in g['genomic_pos_hg19'].keys():
                    genes.loc[IX,k] = (g['genomic_pos_hg19'][k])
        if 'generif' in g.keys():
            genes.loc[IX,'count pubmed'] = len(g['generif'])
        if 'summary' in g.keys():
            genes.loc[IX,'summary'] = g['summary']
    except:
        print()
        print('Exception')
        print('i', i , 'entrez_gene_ID' , entrez_gene_ID, 'kaggle_id', kaggle_id )
        pass
    
    if i%500 == 0:
        print(i, np.round(time.time() - t0 ,1))
    if i >= 10e10:
        break

print(i, 'Finished. Seconds passed: ',  np.round(time.time() - t0 ,1))
        
genes['len'] = genes['end'] - genes['start']
genes

In [ ]:
genes.to_csv('genes_info_MOA_kaggle.csv')


In [ ]:
genes.head(10)


# Show various statistics 

In [ ]:
genes['type_of_gene'].value_counts()

In [ ]:
genes['chr' ].value_counts()

In [ ]:
genes.describe()

In [ ]:
import matplotlib.pyplot as plt

# Distribution on of count publications on genes - "count pubmed"

In [ ]:
plt.figure(figsize = (15,4))
d = genes['count pubmed'].values
plt.plot(np.sort(d)) # genes['count pubmed'].values))
plt.title('Count pubmed')
plt.show()


plt.figure(figsize = (15,4))
d = np.log10( genes['count pubmed'].values )
plt.plot(np.sort(d)) # genes['count pubmed'].values))
plt.title('Log10. Count pubmed')
plt.show()


In [ ]:

plt.figure(figsize = (15,4))
d = ( genes['count pubmed'].values )
plt.hist(d, bins = 50)
plt.title('Log10. Count pubmed')
plt.show()

plt.figure(figsize = (15,4))
d = np.log10( genes['count pubmed'].values )
plt.hist(d, bins = 50 )
plt.title('Log10. Count pubmed')
plt.show()


# Lengths of genes distribution

In [ ]:
c = 'len'
plt.figure(figsize = (15,4))
d = genes[c].values
plt.plot(np.sort(d)) # genes['count pubmed'].values))
plt.title(c)
plt.show()


plt.figure(figsize = (15,4))
d = np.log10( genes[c].values )
plt.plot(np.sort(d)) # genes['count pubmed'].values))
plt.title('Log10. ' + c )
plt.show()

In [ ]:
c = 'len'
plt.figure(figsize = (15,4))
d = ( genes[c].values )
plt.hist(d, bins = 50)
plt.title(c)
plt.show()

plt.figure(figsize = (15,4))
d = np.log10( genes[c].values )
plt.hist(d, bins = 50 )
plt.title('Log10. ' + c )
plt.show()

# Print "summary" info on selected genes

In [ ]:
c = 'len'

g = genes[genes[c].notnull()].sort_values(c, ascending = False)

print('Head of sorted')
for i in range(10): # range(len(genes)):
    try:
        print(i,g['symbol'].iat[i],g['len'].iat[i], g['summary'].iat[i][:180])
    except:
        pass
    
print()
print('Tail of sorted')
for i in range(10): # range(len(genes)):
    i2 = len(g)-i
    try:
        print(i2,g['symbol'].iat[i2],g['len'].iat[i2], g['summary'].iat[i2][:180])
    except:
        pass
    


In [ ]:
c = 'count pubmed' # 'len'

g = genes[genes[c].notnull()].sort_values(c, ascending = False)

print('Head of sorted')
for i in range(10): # range(len(genes)):
    try:
        print(i,g['symbol'].iat[i],g['len'].iat[i],g[c].iat[i], str(g['summary'].iat[i])[:180])
    except:
        pass
    
print()
print('Tail of sorted')
for i in range(10): # range(len(genes)):
    i2 = len(g)-i
    try:
        print(i2,g['symbol'].iat[i2],g['len'].iat[i2],g[c].iat[i2], str(g['summary'].iat[i2])[:180])
    except:
        pass
    


In [ ]:
for i in range(50): # range(len(genes)):
    try:
        print(i,genes['symbol'].iat[i], genes['summary'].iat[i][:120])
    except:
        pass

In [ ]:
for i in range(50): # range(len(genes)):
    i2 = len(genes)-i
    try:
        print(i2,genes['symbol'].iat[i2], genes['summary'].iat[i2][:120])
    except:
        pass